###  先前的版本未考慮到每一個病人不一定符合最大天數的條件，導致個別天數的資料集歸人數量不同
#### 當前處理方式是以最長的輸入時間(e.g:12天)的病人過濾其他天數資料集，已達成每個資料集的歸人數量皆相同
#### 先建構最長天數的資料集後再建構其他的
#### new: 建構預測長度非MS架構的數據，e.g: 8,49 => 2,49 or 4,49

In [3]:
import pandas as pd
import numpy as np
import random as rd
import warnings, glob, os, sys
from tqdm.notebook import tqdm
pd.set_option("display.max_columns", None)
warnings.filterwarnings("ignore")

In [4]:
data_df = pd.read_pickle('./dataset/第一階段處理結果-FinalPeriHDDataFrame.pkl') #Stage1 NLP分類結果
data_df.index[0:5]

Index([1, 2, 3, 4, 5], dtype='int64')

In [5]:
print(data_df.columns.drop(['age','URR%','算年齡','分母體重', '每公斤脫水量(ml/kg)', '有無糖尿病', '處置其他', '症狀處置','sbp', 'dbp','Raw Index', 'New Intra SBP', 'New Intra DBP', 'New 處置其他+症狀處置', 'New 處置其他結束', 'Max Diff mbp', 'Max Diff sbp', 'NLP Judge', 'BP Judge', 'Final Judge']).to_list())

['ID', '洗腎紀錄時間', '洗腎紀錄時間去時分', '急診門診住院', '入院方式', 'FISTULA', 'GRAFT', 'DLC', 'PERMCATH', '處置其他結束', '開始血壓SBP', '開始血壓DBP', '開始脈搏', '體溫', '結束脈搏', 'Catheter', 'K', 'HGB', 'HCT', 'RBC', 'WBC', 'Platelet', 'Creatinine', 'MCV', 'MCHC', 'MCH', 'Na', 'Ca', 'BUN', 'AST (SGOT)', 'ALT (SGPT)', 'Alk.phosphatase', 'Glucose AC', 'ALBUMIN', 'P', 'TIBC', 'Ferritin', 'BUN (洗腎後)', 'IRON/TIBC', 'Intact PTH', 'Iron', 'Bilirubin-T', 'Cholesterol-T', 'CRP', 'HBs', 'HBsAg', 'HBc', 'HCV', '透析液 Ca', 'HBV', '生日', '性別', '體重1開始', '體重2標準體重', '體重結束', '體重實際脫水', '體重機器顯示', '體重3應脫水']


# 特徵篩選

In [4]:
root_path = './dataset/Stage2_EMRs_IDH_csv-05-28_90days-lv3/'
os.makedirs(root_path, exist_ok=True) #創建儲存路徑

In [5]:
# 過濾非生理數據與未使用特徵
data_df.columns
drop_cols = ['洗腎紀錄時間', '急診門診住院', '生日', '算年齡', 
             'DLC', 'PERMCATH', 'HBs', 'HBsAg', 'HBc', 
             
             '體重2標準體重', '體重3應脫水', '體重結束', '分母體重', '體重機器顯示', 
             'BUN (洗腎後)', 
             'sbp', 'dbp', 'New Intra SBP', 'New Intra DBP',
             'New 處置其他+症狀處置', 'New 處置其他結束', '處置其他結束', '處置其他', '症狀處置', 'NLP Judge', 'BP Judge']

# 'Catheter' = 'DLC' + 'PERMCATH'
# 'HBs', 'HBsAg', 'HBc'用於判斷HBV、HCV
# 洗腎後紀錄：'Max Diff mbp'、'Max Diff sbp'、'每公斤脫水量(ml/kg)'、'結束脈搏'
# URR% => BUN、BUN(洗後)
# 可以考慮移除'Bilirubin-T'
select_cols = ['Raw Index', 'ID', '洗腎紀錄時間去時分', 'age', '開始血壓SBP', '開始血壓DBP', '開始脈搏', '體重1開始',
               '入院方式', '性別', '體溫', 'FISTULA', 'GRAFT', 'Catheter', '有無糖尿病','Intact PTH', 'HCV', 'HBV',#catagical data
               'K', 'HGB', 'HCT', 'RBC', 'WBC', 'Platelet',
               'Creatinine', 'MCV', 'MCHC', 'MCH', 'Na', 'Ca', 'BUN', 'AST (SGOT)',
               'ALT (SGPT)', 'Alk.phosphatase', 'Glucose AC', 'ALBUMIN', 'P', 'TIBC',
               'Ferritin', 'IRON/TIBC', 'Iron',
               'Bilirubin-T', 'Cholesterol-T', 'CRP', 'URR%','透析液 Ca', 
               'Max Diff mbp', 'Max Diff sbp', '每公斤脫水量(ml/kg)', '結束脈搏', # 治療後數據
               '體重實際脫水', 'Final Judge',# label: value target, catagical target
               ]
filter_data_df = data_df[select_cols]
# 檢查特徵範圍和遺失值
check_miss_df = {'Feature':[], 'Max':[], 'Min':[], 'Null':[]}
for cols_ in filter_data_df.columns:
    feature = filter_data_df[cols_].values
    check_miss_df["Feature"].append(cols_)
    check_miss_df["Min"].append(min(feature[feature!=9999]))
    check_miss_df["Max"].append(max(feature[feature!=9999]))
    check_miss_df["Null"].append(feature[feature==9999].shape[0])
pd.DataFrame(check_miss_df).to_csv('./dataset/最終數據檢查表.csv', encoding='utf-8-sig', index=False)
save_path = './original_data/'
os.makedirs(os.path.join(root_path, save_path), exist_ok=True) #創建儲存路徑
filter_data_df.to_csv(os.path.join(root_path, save_path,'FinalProcess_df.csv'), encoding='utf-8-sig', index=False)

# 資料分群 - CV [先切Train和Test，Valid從Train中切分]

In [6]:
# 時間序列堆疊，先以groupy方式根據病人ID進行逐筆堆疊
filter_data_df['洗腎紀錄時間去時分'] = pd.to_datetime(filter_data_df['洗腎紀錄時間去時分']).dt.date
filter_data_df['ID'] = filter_data_df['ID'].astype(str)
group_ID_filter_data_df = filter_data_df.groupby(['ID'])
ID_list = filter_data_df['ID'].drop_duplicates().to_list()
total_length = len(ID_list)
train_length = int(0.75 * total_length)
val_length = int(0.15 * total_length)
rd.seed(24)
rd.shuffle(ID_list)
train_list = ID_list[:train_length+val_length]
test_list = ID_list[train_length+val_length:]
print("==初始資料切割狀況==")
train_df = filter_data_df[filter_data_df['ID'].isin(train_list)]
# train_df = train_df[train_df['Timelag']==-1]
print(f"True: {train_df['Final Judge'].sum()} ({round(train_df['Final Judge'].sum()/len(train_df)*100,2)})% False: {len(train_df)-train_df['Final Judge'].sum()} ({round((len(train_df)-train_df['Final Judge'].sum())/len(train_df)*100,2)})%")
test_df = filter_data_df[filter_data_df['ID'].isin(test_list)]
# test_df = test_df[test_df['Timelag']==-1]
print(f"True: {test_df['Final Judge'].sum()} ({round(test_df['Final Judge'].sum()/len(test_df)*100,2)})% False: {len(test_df)-test_df['Final Judge'].sum()} ({round((len(test_df)-test_df['Final Judge'].sum())/len(test_df)*100,2)})%")

==初始資料切割狀況==
True: 36347.0 (15.37)% False: 200167.0 (84.63)%
True: 4702.0 (15.68)% False: 25276.0 (84.32)%


In [7]:
train_df.to_csv(os.path.join(root_path, save_path, 'trainval.csv'), encoding='utf-8-sig', index=False)
test_df.to_csv(os.path.join(root_path, save_path, 'test.csv'), encoding='utf-8-sig', index=False)

In [8]:
filter_data_df.reset_index(drop=True)
# filter_data_df.index = filter_data_df.index+1
filter_data_df.index[0:5]

Index([1, 2, 3, 4, 5], dtype='int64')

# 時序資料處理 Test

In [9]:
# 由於這個步驟全部concat再一起一起做會需要花很多時間，所以先切成歸人csv個別儲存，後續再做合併
# 原本需要花17小時 => 修正後5分鐘
mask_cols = ['Max Diff mbp','Max Diff sbp', 'Final Judge','結束脈搏']
# mask_cols = ['Max Diff mbp','Max Diff sbp','體重實際脫水', 'Final Judge', '每公斤脫水量(ml/kg)','結束脈搏']

ths_days = 30 #change 30 => 90
stack_nums = 12 #max = 12 / 16 lv.3=16, 8, 4
input_nums = 8

In [10]:
for idx__, (group_name, group_data) in enumerate(tqdm(group_ID_filter_data_df)):
    timeseries_df = pd.DataFrame([], columns=filter_data_df.columns)
    timeseries_df['Group Type'] = np.nan #標記Train、Valid、Test
    group_data = group_data.sort_values(by=['洗腎紀錄時間去時分'], ascending=True)
    group_data = group_data.reset_index(drop=True)
    group_data_type = None
    for idx in group_data.index:
        # print(idx)
        try:
            slice_df = group_data.loc[idx:idx+(stack_nums-1)].copy()
            temp_df = slice_df
            if slice_df.shape[0]==stack_nums and (abs(slice_df['洗腎紀錄時間去時分'].iloc[0] - slice_df['洗腎紀錄時間去時分'].iloc[-1]).days<=ths_days): #判斷資料是否有{stack_nums}筆，以及區間時間是否小於等於{ths_days}
                if stack_nums!=input_nums:
                    slice_df = slice_df.iloc[(stack_nums-input_nums)::]
                    slice_df = pd.concat([slice_df,slice_df.iloc[-1].to_frame().T.copy()])
                else: 
                    slice_df = pd.concat([slice_df,slice_df.iloc[-1].to_frame().T.copy()])
                slice_df.iloc[-2,slice_df.columns.get_indexer(mask_cols)] = -9999
                # slice_df = pd.concat([slice_df,slice_df.iloc[-1].to_frame().T.copy()]) #堆疊兩次
                if group_name[0] in test_list:
                    slice_df['Group Type'] = 'Test'
                    group_data_type = 'Test'
                else:
                    slice_df['Group Type'] = 'TrainVal'
                    group_data_type = 'TrainVal'
                timeseries_df = pd.concat([timeseries_df,slice_df])
        except:
            pass
            # break
    # break
    if timeseries_df.shape[0]!=0:
        if stack_nums!=input_nums:
            ts_save_path = os.path.join(root_path,f'time_series_{input_nums-1}_patient_cv')
        else:
            ts_save_path = os.path.join(root_path,f'time_series_{stack_nums-1}_patient_cv')
        os.makedirs(ts_save_path, exist_ok=True) #創建儲存路徑
        timeseries_df.to_csv(os.path.join(ts_save_path,f'{group_data_type}-{str(group_name[0])}.csv'),encoding='utf-8-sig', index=False)
# %%
stack_nums = input_nums

  0%|          | 0/990 [00:00<?, ?it/s]

# 歸人資料合併

In [11]:
filter_data_df = pd.read_csv(os.path.join(root_path, 'original_data', 'FinalProcess_df.csv'))
filter_data_df_cols = filter_data_df.columns
del filter_data_df

patient_csv_glob = glob.glob(os.path.join(root_path, f'time_series_{stack_nums-1}_patient_cv','*'))

save_merge_path = os.path.join(root_path, f'time_series_{stack_nums-1}_cv')
os.makedirs(save_merge_path, exist_ok=True) #創建儲存路徑

In [12]:
# 處理test資料，不需要做cv
test_df = pd.DataFrame([], columns=filter_data_df_cols)
for csv_path in tqdm(patient_csv_glob):
    if 'Test' in csv_path:
        data_df = pd.read_csv(csv_path)
        test_df = pd.concat([test_df,data_df])
        del data_df
test_df.shape

  0%|          | 0/717 [00:00<?, ?it/s]

(258255, 53)

In [13]:
save_name = f'timeseries-{stack_nums-1}.csv'
test_df.to_csv(os.path.join(save_merge_path, 'test_'+save_name), encoding='utf-8-sig', index=False)

In [14]:
trainval_df = pd.DataFrame([], columns=filter_data_df_cols)
for csv_path in tqdm(patient_csv_glob):
    if 'TrainVal' in csv_path:
        data_df = pd.read_csv(csv_path)
        trainval_df = pd.concat([trainval_df,data_df])
        del data_df

  0%|          | 0/717 [00:00<?, ?it/s]

In [15]:
trainval_df = trainval_df.reset_index(drop=True)
trainval_df['ID'] = trainval_df['ID'].astype(str)
trainval_df.index = trainval_df.index+1
print(trainval_df.shape)
get_target_rows_index = [idx_ for idx_ in range(0,len(trainval_df)+stack_nums, (stack_nums+1))][1::]
target_df = trainval_df.loc[get_target_rows_index]
target_df['ID'] = target_df['ID'].astype(str)
print(target_df.shape, len(target_df['ID']))

(2027070, 53)
(225230, 53) 225230


In [16]:
from sklearn.model_selection import StratifiedGroupKFold
sgkf = StratifiedGroupKFold(n_splits=5)
temp_trainval_df = trainval_df.copy()
unique_ID = {value: idx  for idx, value in enumerate(target_df['ID'].unique())}
target_df['group_ID'] = target_df['ID'].apply(lambda x: unique_ID[x])
for fold, (train_index, valid_index) in enumerate(sgkf.split(target_df, target_df['Final Judge'], target_df.group_ID)):
    temp_trainval_df[f'fold_{fold+1}'] = 'Train'
    
    choice_ID = target_df.iloc[train_index]
    print(f"Fold-{fold+1} Train True: {choice_ID['Final Judge'].sum()} ({round(choice_ID['Final Judge'].sum()/len(choice_ID)*100,2)})% False: {len(choice_ID)-choice_ID['Final Judge'].sum()} ({round((len(choice_ID)-choice_ID['Final Judge'].sum())/len(choice_ID)*100,2)})%")
    choice_ID = choice_ID.ID.unique()
    print(f"Patient nums: {len(choice_ID)}")
    
    choice_ID = target_df.iloc[valid_index]
    print(f"Fold-{fold+1} Valid True: {choice_ID['Final Judge'].sum()} ({round(choice_ID['Final Judge'].sum()/len(choice_ID)*100,2)})% False: {len(choice_ID)-choice_ID['Final Judge'].sum()} ({round((len(choice_ID)-choice_ID['Final Judge'].sum())/len(choice_ID)*100,2)})%")
    choice_ID = choice_ID.ID.unique()
    print(f"Patient nums: {len(choice_ID)}")
    valid_df_idx = trainval_df[trainval_df['ID'].isin(choice_ID)].copy().index
    
    temp_trainval_df.loc[valid_df_idx,f'fold_{fold+1}'] = 'Valid'

# if stack_nums!=12:
#     filter_ID_train = pd.read_csv('./dataset/Stage2_EMRs_IDH_csv-05-28/time_series_11_cv/trainval_timeseries-11.csv').astype(str)['ID'].unique()
#     temp_trainval_df = temp_trainval_df[temp_trainval_df['ID'].str.isin(filter_ID_train.tolist())]

save_name = f'trainval_timeseries-{stack_nums-1}.csv'
print(temp_trainval_df.shape)
temp_trainval_df.to_csv(os.path.join(save_merge_path, save_name), encoding='utf-8-sig', index=False)

Fold-1 Train True: 28091.0 (15.58)% False: 152163.0 (84.42)%
Patient nums: 510
Fold-1 Valid True: 6937.0 (15.42)% False: 38039.0 (84.58)%
Patient nums: 135
Fold-2 Train True: 28043.0 (15.55)% False: 152245.0 (84.45)%
Patient nums: 523
Fold-2 Valid True: 6985.0 (15.54)% False: 37957.0 (84.46)%
Patient nums: 122
Fold-3 Train True: 28056.0 (15.56)% False: 152231.0 (84.44)%
Patient nums: 522
Fold-3 Valid True: 6972.0 (15.51)% False: 37971.0 (84.49)%
Patient nums: 123
Fold-4 Train True: 27917.0 (15.51)% False: 152092.0 (84.49)%
Patient nums: 521
Fold-4 Valid True: 7111.0 (15.72)% False: 38110.0 (84.28)%
Patient nums: 124
Fold-5 Train True: 28005.0 (15.55)% False: 152077.0 (84.45)%
Patient nums: 504
Fold-5 Valid True: 7023.0 (15.56)% False: 38125.0 (84.44)%
Patient nums: 141
(2027070, 58)


In [17]:
# from sklearn.model_selection import StratifiedGroupKFold
# sgkf = StratifiedGroupKFold(n_splits=5)
# # target_df['group_counts']
# unique_ID = {value: idx  for idx, value in enumerate(target_df['ID'].unique())}
# target_df['group_ID'] = target_df['ID'].apply(lambda x: unique_ID[x])
# for fold, (train_index, valid_index) in enumerate(sgkf.split(target_df, target_df['Final Judge'], target_df.group_ID)):
    
#     choice_ID = target_df.iloc[train_index]
#     print(f"Fold-{fold+1} Train True: {choice_ID['Final Judge'].sum()} ({round(choice_ID['Final Judge'].sum()/len(choice_ID)*100,2)})% False: {len(choice_ID)-choice_ID['Final Judge'].sum()} ({round((len(choice_ID)-choice_ID['Final Judge'].sum())/len(choice_ID)*100,2)})%")
#     choice_ID = choice_ID.ID.unique()
#     print(f"Patient nums: {len(choice_ID)}")
#     train_df = trainval_df[trainval_df['ID'].isin(choice_ID)]
#     train_df['Group Type'] = 'Train'    
#     print(train_df.shape)
    
#     choice_ID = target_df.iloc[valid_index]
#     print(f"Fold-{fold+1} Valid True: {choice_ID['Final Judge'].sum()} ({round(choice_ID['Final Judge'].sum()/len(choice_ID)*100,2)})% False: {len(choice_ID)-choice_ID['Final Judge'].sum()} ({round((len(choice_ID)-choice_ID['Final Judge'].sum())/len(choice_ID)*100,2)})%")
#     choice_ID = choice_ID.ID.unique()
#     print(f"Patient nums: {len(choice_ID)}")
#     valid_df = trainval_df[trainval_df['ID'].isin(choice_ID)]
#     valid_df['Group Type'] = 'Valid'    
#     print(valid_df.shape)
    
#     concat_train_val = pd.concat([train_df, valid_df])
#     save_name = f'fold_{fold+1}_trainval_timeseries-{stack_nums-1}.csv'
#     print(concat_train_val.shape)
#     concat_train_val.to_csv(os.path.join(save_merge_path, save_name), encoding='utf-8-sig', index=False)
#     # break